# 11_패딩(Padding)
자연어 처리를 하다 보면 각 문장 혹은 문서의 길이가 서로 다른 경우가 있다. 기계는 길이가 동일한 문서들에 대해 하나의 행렬로 간주하고, 한꺼번에 묶어 처리(병렬연산)한다. 이는 연산의 속도와 효율성을 높일 수 있는 방법이기에, 이를 위해 여러 문장의 길이를 임의로 같게 맞추어 주는 작업이 필요하다. 실습을 통해 이해해보자.

## 1. Numpy로 패딩하기

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

정수 인코딩에서 수행했던 실습을 반복해보도록 한다. 아래와 같은 텍스트 데이터를 생성한다.

In [2]:
sentences = [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]

정수 인코딩을 수행한다. 

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences) # fit_on_texts() 안에 코퍼스를 입력하면 빈도수를 기준으로 단어 집합을 생성

이에 따라 텍스트의 모든 단어들은 빈도수의 수에 따라 순위가 매겨지게 되고, 각 단어들에는 그 순위와 같은 정수가 맵핑된다. 이 정수들을 위의 텍스트 데이터와 치환한 것이 정수 인코딩이며, 아래의 출력이다.   

In [4]:
encoded = tokenizer.texts_to_sequences(sentences)
print(encoded)

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


모든 단어가 고유한 정수로 변환된 것을 확인할 수 있다. 이제 각 데이터를 동일한 길이로 맞추어 주기 위해 위의 데이터 중 가장 길이가 긴 문장의 길이를 계산한다.

In [5]:
max_len = max(len(item) for item in encoded)
print(max_len)

7


가장 길이가 긴 문장의 길이는 7이다. 이제 모든 문장의 길이를 7로 맞춰준다. 이 때는 가상의 단어 'PAD'를 사용한다. 'PAD'라는 단어가 있다고 가정하고, 이 단어는 0번 단어라고 정의한다. 이제 길이가 7보다 짧은 문장에는 숫자 0을 채워서 전부 길이 7로 맞춰준다. 

In [6]:
for item in encoded: # 각 문장에 대해서
    while len(item) < max_len: # max_len보다 길이가 짧을 경우
        item.append(0)
        
padded_np = np.array(encoded)
padded_np

array([[ 1,  5,  0,  0,  0,  0,  0],
       [ 1,  8,  5,  0,  0,  0,  0],
       [ 1,  3,  5,  0,  0,  0,  0],
       [ 9,  2,  0,  0,  0,  0,  0],
       [ 2,  4,  3,  2,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  2,  0,  0,  0,  0],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0,  0,  0]])

길이가 7보다 짧은 문장에는 모자란 만큼 숫자 0이 붙어 길이가 7로 균일하게 맞춰진 것을 볼 수 있다. 기계는 이제 이러한 데이터들을 하나의 행렬로 보고, 병렬 처리를 할 수 있게 된다. 또한 0번 단어는 아무런 의미도 없는 단어이므로 자연어 처리 과정에서 기계는 이를 무시하게 된다. 
<br>

이와 같이 데이터에 특정값을 채워 데이터의 크기(shape)을 조정하는 것을 패딩(padding)이라고 한다. 숫자 0이 특정값일 경우, 이를 제로 패딩(zero padding)이라 한다. 

## 2. 케라스 전처리 도구로 패딩하기

케라스에서는 위와 같은 패딩을 위한 도구인 pad_sequences를 제공한다. 

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

앞에서 encoded 값이 이미 패딩한 후의 결과로 저장되어 있으므로, 패딩 이전의 값으로 되돌린다. 

In [8]:
encoded = tokenizer.texts_to_sequences(sentences)
print(encoded)

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


케라스의 pad_sequences를 사용해 패딩한다. 

In [9]:
padded = pad_sequences(encoded)
padded

array([[ 0,  0,  0,  0,  0,  1,  5],
       [ 0,  0,  0,  0,  1,  8,  5],
       [ 0,  0,  0,  0,  1,  3,  5],
       [ 0,  0,  0,  0,  0,  9,  2],
       [ 0,  0,  0,  2,  4,  3,  2],
       [ 0,  0,  0,  0,  0,  3,  2],
       [ 0,  0,  0,  0,  1,  4,  6],
       [ 0,  0,  0,  0,  1,  4,  6],
       [ 0,  0,  0,  0,  1,  4,  2],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 0,  0,  0,  1, 12,  3, 13]])

Numpy로 패딩을 진행했을 때와는 패딩 결과가 다르다. pad_sequences는 기본적으로 문서의 뒤에 0을 채우는 것이 아니라 앞을 0으로 채우기 때문이다. 앞서 했던 패딩처럼 0으로 뒤를 채우고 싶다면 인자로 padding='post'를 주면 된다.

In [10]:
padded = pad_sequences(encoded, padding='post')
padded

array([[ 1,  5,  0,  0,  0,  0,  0],
       [ 1,  8,  5,  0,  0,  0,  0],
       [ 1,  3,  5,  0,  0,  0,  0],
       [ 9,  2,  0,  0,  0,  0,  0],
       [ 2,  4,  3,  2,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  2,  0,  0,  0,  0],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0,  0,  0]])

Numpy를 이용하여 패딩을 했을 때와 결과가 동일하다. 실제로 결과가 동일한지 두 결과를 비교해본다. 

In [11]:
(padded == padded_np).all()

True

True 값이 리턴된다. 두 결과가 동일하다는 의미다. 지금까지는 가장 긴 길이를 가진 문서를 기준으로 패딩을 한다고 가정했다. 하지만 꼭 그를 기준으로 해야 하는 것은 아니다. 가령, 모든 문서의 평균 길이가 20인데 문서 1개의 길이가 5,000이라고 해서 굳이 모든 문서의 길이를 5,000으로 패딩할 필요는 없을 수 있는 것이다. 이와 같은 경우, 길이에 제한을 두고 패딩할 수 있다. max_len의 인자로 정수를 주면, 해당 정수로 모든 문서의 길이를 동일하게 할 수 있다.

In [12]:
padded = pad_sequences(encoded, padding='post', maxlen= 5)
padded

array([[ 1,  5,  0,  0,  0],
       [ 1,  8,  5,  0,  0],
       [ 1,  3,  5,  0,  0],
       [ 9,  2,  0,  0,  0],
       [ 2,  4,  3,  2,  0],
       [ 3,  2,  0,  0,  0],
       [ 1,  4,  6,  0,  0],
       [ 1,  4,  6,  0,  0],
       [ 1,  4,  2,  0,  0],
       [ 3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0]])

길이가 5보다 짧은 문서들은 0으로 패딩되고, 기존에 5보다 길었다면 **데이터가 손실된다**. 제로 패딩은 널리 퍼진 관례이지만, 반드시 지켜야 하는 규칙은 아니다. 만약, 숫자 0이 아니라 다른 숫자를 특정값으로 활용하는 것도 가능하다. 현재 사용된 정수들과 겹치지 않도록, 단어 집합의 크기에 +1을 해보자. 

In [13]:
last_value = len(tokenizer.word_index) + 1
print(last_value)

14


현재 단어가 총 13개이고, 1번부터 13번까지 정수가 사용되었기 떄문에 단어 집합의 크기에 +1을 하면 마지막 숫자인 13보다 1이 큰 14를 얻는다. pad_sequences의 인자로 value를 사용하면 0이 아닌 다른 숫자로 패딩이 가능하다.

In [14]:
padded = pad_sequences(encoded, padding='post', value= last_value)
padded

array([[ 1,  5, 14, 14, 14, 14, 14],
       [ 1,  8,  5, 14, 14, 14, 14],
       [ 1,  3,  5, 14, 14, 14, 14],
       [ 9,  2, 14, 14, 14, 14, 14],
       [ 2,  4,  3,  2, 14, 14, 14],
       [ 3,  2, 14, 14, 14, 14, 14],
       [ 1,  4,  6, 14, 14, 14, 14],
       [ 1,  4,  6, 14, 14, 14, 14],
       [ 1,  4,  2, 14, 14, 14, 14],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13, 14, 14, 14]])